In [1]:
from vectorbtpro import *
import pandas as pd
import numpy as np

vbt.settings.set_theme("dark")
vbt.settings.plotting["layout"]["width"] = 800
vbt.settings.plotting['layout']['height'] = 200
vbt.settings.plotting.use_resampler = True # Need to pip install https://github.com/predict-idlab/plotly-resampler

pd.set_option('display.max_rows', None)

# Type out the version of vectorbtpro Note for this instance I'm using 2024.2.22 note it has a different way of importing Vectorbt see release notes https://vectorbt.pro/pvt_321460c7/getting-started/release-notes/
vbt.__version__

'2024.2.22'

In [53]:
# filename = 'trade history - 12.2023.xlsx'
# filename = 'data/Crypto-to-the-moon-orders.csv'
filename = 'data/Average-Moon-Cypress-orders.csv'
trades = pd.read_csv(filename)

trades['open_date'] = pd.to_datetime(trades['open_date'])
trades['close_date'] = pd.to_datetime(trades['closed_date'])
trades['symbol'] = trades['title'].str.extract(r'(\w+)')
trades['trade_type'] = trades['leverage'].apply(lambda x: 'open_long' if 'Long' in x else 'close_long' if 'Long' in x else 'open_short' if 'Short' in x else 'close_short')
trades = trades.drop(columns=['Unnamed: 0'])
# Function to clean and convert currency columns
def clean_currency_column(column):
    return pd.to_numeric(column.str.replace(',', '').str.replace(' USDT', ''), errors='coerce')

# Clean the currency columns
trades['entry_price'] = clean_currency_column(trades['entry_price'])
trades['pnl'] = clean_currency_column(trades['pnl'])
trades['fill_price'] = clean_currency_column(trades['fill_price'])

# Extracting the number of contracts
trades['num_contracts'] = trades['closed'].str.extract('(\d+,?\d*)').replace(',', '', regex=True).astype(int)

# Get a list of all unique symbols
symbols = trades['symbol'].unique()
# Look up the contract multiplier for each symbol from the exchange website https://www.okx.com/trade-market/info/swap
symbols_dict_contract_multiplier = {'ETHUSDT':0.1, 'BTCUSDT':0.01, 'PEOPLEUSDT':100, 'ORDIUSDT':0.1, 'SOLUSDT':1,
       'DOGEUSDT':1000, 'USTCUSDT':100, 'BNBUSDT':0.01}
trades['contract_multiplier'] = trades['symbol'].map(symbols_dict_contract_multiplier)
trades['quantity'] = trades['num_contracts'] * trades['contract_multiplier']

# Now separate out the orders into open and closing orders and sort by date
# Open Orders
open_orders = trades[['open_date', 'title', 'direction', 'leverage', 'entry_price', 'symbol', 'num_contracts', 'quantity']].copy()
open_orders.rename(columns={'open_date': 'date'}, inplace=True)
open_orders['price'] = open_orders['entry_price']
open_orders['trade_type'] = open_orders['leverage'].apply(lambda x: 'open_long' if 'Long' in x else 'open_short')

# Closing Orders
closing_orders = trades[['closed_date', 'title', 'direction', 'leverage', 'fill_price', 'pnl', 'pnl_percent', 'symbol', 'num_contracts', 'quantity']].copy()
closing_orders.rename(columns={'closed_date': 'date', 'fill_price': 'close_price'}, inplace=True)
closing_orders['price'] = closing_orders['close_price']
closing_orders['trade_type'] = closing_orders['leverage'].apply(lambda x: 'close_long' if 'Long' in x else 'close_short')

# Combine the two dataframes
orders = pd.concat([open_orders, closing_orders])

# Convert date columns to datetime for sorting
orders['date'] = pd.to_datetime(orders['date'], errors='coerce')

# Sorting by date
orders = orders.sort_values(by='date')
orders.set_index('date', inplace=True)
# Localize the index to Central Standard Time (CST) first
orders.index = orders.index.tz_localize('America/Chicago')
# Then convert the timezone from CST to UTC
orders.index = orders.index.tz_convert('UTC')
# Revised approach to handle duplicate timestamps by adding milliseconds
def add_milliseconds_to_duplicates(df):
    # Create a new Series to hold adjusted timestamps
    adjusted_timestamps = []

    # Create a dictionary to track the count of each timestamp
    timestamp_count = {}

    # Iterate through each timestamp in the index
    for timestamp in df.index:
        # If the timestamp is not in the dictionary, add it with a count of 0
        if timestamp not in timestamp_count:
            timestamp_count[timestamp] = 0
            adjusted_timestamps.append(timestamp)
        else:
            # If the timestamp is already in the dictionary, increment the count
            timestamp_count[timestamp] += 1
            # Add milliseconds to the timestamp based on its count
            new_timestamp = timestamp + pd.Timedelta(milliseconds=timestamp_count[timestamp])
            adjusted_timestamps.append(new_timestamp)

    return pd.Series(adjusted_timestamps, index=df.index)

# # Apply the function to adjust the index
adjusted_index = add_milliseconds_to_duplicates(orders)
orders.index = adjusted_index
orders = orders.sort_index()
# If open-short and close-long then quantity should be negative else positive
orders['trade_direction'] = orders.apply(lambda x: -1 if (x['trade_type'] == 'open_short' or x['trade_type'] == 'close_long') else 1, axis=1)
orders['trade_quantity'] = orders['trade_direction'] * orders['quantity']

display(symbols)
orders.tail(20)
# orders.tail()

array(['SOLUSDT', 'ETHUSDT', 'BTCUSDT', 'APEUSDT', 'UNIUSDT', 'ETCUSDT',
       'LINKUSDT', 'ORDIUSDT', 'DOGEUSDT', 'ARBUSDT', 'OPUSDT',
       'SHIBUSDT', 'SUIUSDT', 'WAVESUSDT', 'AVAXUSDT', 'FTMUSDT',
       'BNBUSDT'], dtype=object)

,title,direction,leverage,entry_price,symbol,num_contracts,quantity,price,trade_type,close_price,pnl,pnl_percent,trade_direction,trade_quantity
2024-02-20 11:08:03+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,NaN,ETHUSDT,200,20.0,2931.60,close_long,2931.60,276.99,+9.49%,-1,-20.0
2024-02-20 11:08:03.001000+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,NaN,ETHUSDT,200,20.0,2931.34,close_long,2931.34,626.81,+21.62%,-1,-20.0
2024-02-20 11:08:19+00:00,BTCUSDT Perpetual,Cross,Long 10.00x,52275.60,BTCUSDT,400,4.0,52275.60,open_long,NaN,NaN,NaN,1,4.0
2024-02-20 12:26:44+00:00,BTCUSDT Perpetual,Cross,Long 10.00x,NaN,BTCUSDT,400,4.0,52331.40,close_long,52331.40,98.02,+0.46%,-1,-4.0
2024-02-20 13:35:50+00:00,SOLUSDT Perpetual,Cross,Long 10.00x,110.04,SOLUSDT,300,300.0,110.04,open_long,NaN,NaN,NaN,1,300.0
2024-02-20 14:42:03+00:00,SOLUSDT Perpetual,Cross,Long 10.00x,107.27,SOLUSDT,200,200.0,107.27,open_long,NaN,NaN,NaN,1,200.0
2024-02-20 15:24:19+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,2931.97,ETHUSDT,400,40.0,2931.97,open_long,NaN,NaN,NaN,1,40.0
2024-02-20 15:30:02+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,NaN,ETHUSDT,400,40.0,2910.61,close_long,2910.61,-924.51,-15.77%,-1,-40.0
2024-02-20 15:36:28+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,2911.02,ETHUSDT,400,40.0,2911.02,open_long,NaN,NaN,NaN,1,40.0
2024-02-20 17:33:17+00:00,ETHUSDT Perpetual,Cross,Long 20.00x,NaN,ETHUSDT,400,40.0,2903.81,close_long,2903.81,-383.80,-6.60%,-1,-40.0


In [42]:
# trades.to_csv('data/Crypto-to-the-moon-orders-cleaned.csv')

# Download minutely data from Binance
Uncomment the next cell if you don't already have the data

In [54]:
from datetime import datetime
min_date = trades['open_date'].min().floor('D')
max_date = trades['close_date'].max().ceil('D')

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")


Minimum date: 2023-11-21 00:00:00
Maximum date: 2024-02-22 00:00:00


In [69]:
symbols
# if SHIBUSDT in `symbols`: change it to '1000SHIBUSDT' this is a binance specific issue
symbols = ['1000SHIBUSDT' if x == 'SHIBUSDT' else x for x in symbols]
symbols


['SOLUSDT',
 'ETHUSDT',
 'BTCUSDT',
 'APEUSDT',
 'UNIUSDT',
 'ETCUSDT',
 'LINKUSDT',
 'ORDIUSDT',
 'DOGEUSDT',
 'ARBUSDT',
 'OPUSDT',
 '1000SHIBUSDT',
 'SUIUSDT',
 'WAVESUSDT',
 'AVAXUSDT',
 'FTMUSDT',
 'BNBUSDT']

In [70]:
data = vbt.BinanceData.pull(symbols, start=min_date, end=max_date, timeframe='15T', klines_type='futures', timezone='UTC')
data.save('price_data.pkl')
data = vbt.BinanceData.load('price_data.pkl')
data['Close'].get().tail()

  0%|          | 0/17 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

symbol,SOLUSDT,ETHUSDT,BTCUSDT,APEUSDT,UNIUSDT,ETCUSDT,LINKUSDT,ORDIUSDT,DOGEUSDT,ARBUSDT,OPUSDT,1000SHIBUSDT,SUIUSDT,WAVESUSDT,AVAXUSDT,FTMUSDT,BNBUSDT
Open time,,,,,,,,,,,,,,,,,
2024-02-20 22:45:00+00:00,108.921,2997.40,52194.3,1.8270,7.573,27.049,19.374,67.782,0.08601,1.9837,4.0421,0.009753,1.7327,2.3514,38.573,0.4262,354.96
2024-02-20 23:00:00+00:00,109.482,3022.30,52426.5,1.8390,7.590,27.266,19.452,68.551,0.08639,2.0020,4.0796,0.009794,1.7384,2.3593,38.717,0.4277,356.51
2024-02-20 23:15:00+00:00,108.925,3014.78,52355.8,1.8301,7.560,27.137,19.385,67.865,0.08621,2.0204,4.0831,0.009768,1.7270,2.3564,38.581,0.4259,355.55
2024-02-20 23:30:00+00:00,108.671,3010.72,52259.8,1.8417,7.570,27.074,19.317,67.735,0.08635,2.0089,4.0706,0.009763,1.7264,2.3499,38.568,0.4256,356.18
2024-02-20 23:45:00+00:00,108.418,3017.21,52297.4,1.8479,7.565,27.089,19.296,67.527,0.08618,2.0118,4.0702,0.009765,1.7189,2.3520,38.523,0.4246,354.86


In [80]:
# Choose a symbol to analyze
symbol = 'ORDIUSDT'

symbol_data = data['Close'].get(symbol=symbol)
symbol_orders = orders[orders['symbol'] == symbol]

# Identify duplicates in the index
symbol_orders.index[symbol_orders.index.duplicated()]
# Throw an error if there are duplicates
if symbol_orders.index.duplicated().any():
    raise ValueError('Duplicate timestamps found in the orders dataframe')

# Change Floor and Ceiling based on your preference to zoom in or out. This is just rounding the date to the nearest hour/day/etc.
min_date = symbol_orders.index.min().floor('D') 
max_date = symbol_orders.index.max().ceil('D')
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

unlevered_pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    # fixed_fees = trades['fees'],
    init_cash = 'auto', #40000,
    # leverage=10,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)
print(f'Unlevered Portfolio Sim Sharpe Ratio: {unlevered_pf.sharpe_ratio}')
init_cash = unlevered_pf.init_cash 
leverage = 10
pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    # fixed_fees = trades['fees'],
    init_cash = init_cash/leverage,
    leverage=leverage,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)

display(pf.stats())
pf.trades.records_readable
pf.plot().show()

Minimum date: 2023-11-22 00:00:00+00:00
Maximum date: 2024-02-14 00:00:00+00:00
Unlevered Portfolio Sim Sharpe Ratio: 3.2816285570936223


Start                         2023-11-22 00:00:00+00:00
End                           2024-02-14 00:00:00+00:00
Period                                 84 days 09:15:00
Start Value                                  7592.29015
Min Value                                    7365.29015
Max Value                                   23937.91005
End Value                                   23560.91005
Total Return [%]                             210.326787
Benchmark Return [%]                         249.044553
Total Time Exposure [%]                       10.739415
Max Gross Exposure [%]                       784.651656
Max Drawdown [%]                              34.178473
Max Drawdown Duration                  41 days 03:45:00
Total Orders                                         36
Total Fees Paid                                     0.0
Total Trades                                         18
Win Rate [%]                                  88.888889
Best Trade [%]                                14

# Now let's look at the entire portfolio

let's look at a single asset like BTC but make the code less rigid

In [32]:
print(symbols)

['ETHUSDT' 'BTCUSDT' 'PEOPLEUSDT' 'ORDIUSDT' 'SOLUSDT' 'DOGEUSDT'
 'USTCUSDT' 'BNBUSDT']


In [57]:
# Choose a symbol to analyze
symbol = 'BTCUSDT'

symbol_data = data['Close'].get(symbol=symbol)
symbol_orders = orders[orders['symbol'] == symbol]

# Identify duplicates in the index
symbol_orders.index[symbol_orders.index.duplicated()]
# Throw an error if there are duplicates
if symbol_orders.index.duplicated().any():
    raise ValueError('Duplicate timestamps found in the orders dataframe')

# Change Floor and Ceiling based on your preference to zoom in or out. This is just rounding the date to the nearest hour/day/etc.
min_date = symbol_orders.index.min().floor('D') 
max_date = symbol_orders.index.max().ceil('D')
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

unlevered_pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    # fixed_fees = trades['fees'],
    init_cash = 'auto', #40000,
    # leverage=10,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)

init_cash = unlevered_pf.init_cash 
leverage = 10
pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    # fixed_fees = trades['fees'],
    init_cash = init_cash/leverage,
    leverage=leverage,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)

display(pf.stats())
pf.trades.records_readable
pf.plot().show()

Minimum date: 2023-11-21 00:00:00+00:00
Maximum date: 2024-02-21 00:00:00+00:00


Start                         2023-11-21 00:00:00+00:00
End                           2024-02-20 23:45:00+00:00
Period                                 94 days 19:15:00
Start Value                                   71394.892
Min Value                                     70277.192
Max Value                                    123632.852
End Value                                     97990.552
Total Return [%]                              37.251489
Benchmark Return [%]                          39.495444
Total Time Exposure [%]                       38.325459
Max Gross Exposure [%]                       911.657391
Max Drawdown [%]                              25.493467
Max Drawdown Duration                  21 days 13:15:00
Total Orders                                        270
Total Fees Paid                                     0.0
Total Trades                                        135
Win Rate [%]                                   50.37037
Best Trade [%]                                 5

In [ ]:
display(pf.orders.records_readable)
display(pf.trades.records_readable)

eyeball check to make sure the original data and the portfolio simulation are similar

In [ ]:
symbol_orders

# Old Code 👇👇👇👇👇

If you want to hide the benchmark you can just set `bm_returns=False`

In [232]:
default_time_in_market = float(elk_pf.stats(metrics="total_time_exposure").iloc[0])
print(f'The total time in market as calculated by VBT default (Not Capital Weighted): {default_time_in_market:.2f}%')
# Assuming 'portfolio_trades' is your vbt.Portfolio object
portfolio_trades = elk_pf.trades.records_readable

# Calculate the time each trade is open
portfolio_trades['trade_duration'] = (portfolio_trades['Exit Index'] - portfolio_trades['Entry Index'])


# Calculate the capital invested in each trade (assuming 'Size' is the quantity and 'Avg Entry Price' is the price)
portfolio_trades['capital_invested'] = portfolio_trades['Size'] * portfolio_trades['Avg Entry Price']
print(f'The sum of all trades capital invested: {portfolio_trades.capital_invested.sum()}')
print(f'The sum of all the time in market Not Capital Weighted: {portfolio_trades["trade_duration"].sum()}')
# print(portfolio_trades['trade_duration'].sum().total_seconds()/(60*60*24))
# Calculate the total time for the portfolio
total_time = elk_pf.wrapper.index[-1]-elk_pf.wrapper.index[0]
print(f'The total time is: {total_time}')
# Calculate the weighted time
weighted_time = (portfolio_trades['trade_duration'] * portfolio_trades['capital_invested']).sum()
print(f'The capital weighted time in seconds is: {weighted_time.total_seconds():.0f}')

print(f'Total Time in seconds multiplied by Total Capital Invested {total_time.total_seconds() * portfolio_trades["capital_invested"].sum():.0f}')
print(f'Total Time in seconds multiplied by Initial Capital {total_time.total_seconds() * elk_pf.init_cash:.0f}')
print(f'Total Time in seconds multiplied by Average Capital {total_time.total_seconds() * elk_pf.value.mean():.0f}')

# Calculate the capital weighted time in market as a percentage of the total time multiplied byt the initial capital invested Could use average capital invested instead
# capital_weighted_time_pct = (weighted_time.total_seconds() / (total_time.total_seconds() * elk_pf.init_cash)) * 100 
# If you want to use the average capital balance instead of the initial capital
capital_weighted_time_pct = (weighted_time.total_seconds() / (total_time.total_seconds() * elk_pf.value.mean())) * 100
print(f'Beginning capital ${elk_pf.init_cash:.2f} Ending capital ${elk_pf.value[-1]:.2f} Average capital ${elk_pf.value.mean():.2f}')
print('\nCapital Weighted Time in Market divided by total time multiplied by average capital:')
print(f"Capital Weighted Time in Market [%]: {float(capital_weighted_time_pct):.2f}%")




The total time in market as calculated by VBT default (Not Capital Weighted): 8.03%
The sum of all trades capital invested: 53691.98181
The sum of all the time in market Not Capital Weighted: 4 days 18:44:19.813000
The total time is: 31 days 02:05:00.678000
The capital weighted time in seconds is: 237522961
Total Time in seconds multiplied by Total Capital Invested 144211330347
Total Time in seconds multiplied by Initial Capital 6714751695
Total Time in seconds multiplied by Average Capital 6906874052
Beginning capital $2500.00 Ending capital $2625.33 Average capital $2571.53

Capital Weighted Time in Market divided by total time multiplied by average capital:
Capital Weighted Time in Market [%]: 3.44%


### Let's create a custom metric based on Capital Weighted Time Exposure [%]

In [294]:
elk_pf.stats()

Start                                        2023-12-01 00:00:00+00:00
End                                   2024-01-01 02:05:00.678000+00:00
Period                                                31 days 02:46:00
Start Value                                                     2500.0
Min Value                                                   2484.71059
Max Value                                                  2632.431622
End Value                                                  2625.328886
Total Return [%]                                              5.013155
Benchmark Return [%]                                         11.257526
Total Time Exposure [%]                                       8.030175
Max Gross Exposure [%]                                      179.120696
Max Drawdown [%]                                              0.955434
Max Drawdown Duration                                  3 days 11:03:00
Total Orders                                                       166
Total 

In [295]:
max_winning_streak

('max_winning_streak',
 {'title': 'Max Winning Streak', 'calc_func': 'trades.winning_streak.max'})

In [251]:
def calc_capital_weighted_time_in_market(portfolio):
    '''Calculates the capital weighted time in market as a percentage of the total time multiplied by the average capital invested'''
    portfolio_trades = portfolio.trades.records_readable
    trade_duration = portfolio_trades['Exit Index'] - portfolio_trades['Entry Index']
    capital_invested = portfolio_trades.Size * portfolio_trades['Avg Entry Price']
    weighted_time = (trade_duration * capital_invested).sum()
    total_time = portfolio.wrapper.index[-1]-portfolio.wrapper.index[0]
    capital_weighted_time_pct = (weighted_time.total_seconds() / (total_time.total_seconds() * portfolio.value.mean())) * 100
    return capital_weighted_time_pct

print(f'Capital Weighted Time Invested [%]: {calc_capital_weighted_time_in_market(elk_pf):.2f}%')

# Example of how to create a custom metric
# https://vectorbt.dev/api/portfolio/base/#custom-metrics
max_winning_streak = (
    'max_winning_streak',
    dict(
        title='Max Winning Streak',
        calc_func='trades.winning_streak.max'
    )
)

max_losing_streak = (
    'max_losing_streak',
    dict(
        title='Max Losing Streak',
        calc_func='trades.losing_streak.max'
    )
)

capital_weighted_time_exposure = (
    'capital_weighted_time_exposure',
    dict(
        title='Capital Weighted Time Exposure [%]',
        calc_func=lambda self, group_by:
        calc_capital_weighted_time_in_market(self)
    )
)

display(elk_pf.stats(metrics=capital_weighted_time_exposure))
display(elk_pf.stats(metrics=max_winning_streak))
display(elk_pf.stats(metrics=max_losing_streak))
# Add the custom metric to the portfolio stats as the last item
elk_pf.metrics['max_winning_streak'] = max_winning_streak[1]
elk_pf.metrics['max_losing_streak'] = max_losing_streak[1]
elk_pf.stats()

Capital Weighted Time Invested [%]: 3.44%


Capital Weighted Time Exposure [%]    3.438936
dtype: object

Max Winning Streak    40.0
dtype: object

Max Losing Streak    3.0
dtype: object

Start                                        2023-12-01 00:00:00+00:00
End                                   2024-01-01 02:05:00.678000+00:00
Period                                                31 days 02:46:00
Start Value                                                     2500.0
Min Value                                                   2484.71059
Max Value                                                  2632.431622
End Value                                                  2625.328886
Total Return [%]                                              5.013155
Benchmark Return [%]                                         11.257526
Total Time Exposure [%]                                       8.030175
Max Gross Exposure [%]                                      179.120696
Max Drawdown [%]                                              0.955434
Max Drawdown Duration                                  3 days 11:03:00
Total Orders                                                       166
Total 

In [281]:
# Example of how to create a custom metric
# https://vectorbt.dev/api/portfolio/base/#custom-metrics

print(elk_pf.stats(metrics=capital_weighted_time_exposure))
# Add the custom metric to the portfolio stats as the last item
elk_pf.metrics['max_winning_streak'] = max_winning_streak[1]
elk_pf.metrics['capital_weighted_time_exposure'] = capital_weighted_time_exposure[1]
elk_pf.stats()

Capital Weighted Time Exposure [%]    3.438936
dtype: object


Start                                        2023-12-01 00:00:00+00:00
End                                   2024-01-01 02:05:00.678000+00:00
Period                                                31 days 02:46:00
Start Value                                                     2500.0
Min Value                                                   2484.71059
Max Value                                                  2632.431622
End Value                                                  2625.328886
Total Return [%]                                              5.013155
Benchmark Return [%]                                         11.257526
Total Time Exposure [%]                                       8.030175
Max Gross Exposure [%]                                      179.120696
Max Drawdown [%]                                              0.955434
Max Drawdown Duration                                  3 days 11:03:00
Total Orders                                                       166
Total 

In [286]:
my_metrics = list(elk_pf.metrics.items())
print(my_metrics)
# Now set the metrics globally under `portfolio.stats` in settings
vbt.settings.portfolio['stats']['metrics'] = my_metrics

[('start', {'title': 'Start', 'calc_func': <function Portfolio.<lambda> at 0x7fa6196bd750>, 'agg_func': None, 'tags': 'wrapper'}), ('end', {'title': 'End', 'calc_func': <function Portfolio.<lambda> at 0x7fa6196bd7e0>, 'agg_func': None, 'tags': 'wrapper'}), ('period', {'title': 'Period', 'calc_func': <function Portfolio.<lambda> at 0x7fa6196bd870>, 'apply_to_timedelta': True, 'agg_func': None, 'tags': 'wrapper'}), ('start_value', {'title': 'Start Value', 'calc_func': 'init_value', 'tags': 'portfolio'}), ('min_value', {'title': 'Min Value', 'calc_func': 'value.vbt.min', 'tags': 'portfolio'}), ('max_value', {'title': 'Max Value', 'calc_func': 'value.vbt.max', 'tags': 'portfolio'}), ('end_value', {'title': 'End Value', 'calc_func': 'final_value', 'tags': 'portfolio'}), ('cash_deposits', {'title': 'Cash Deposits', 'calc_func': 'cash_deposits.vbt.sum', 'check_has_cash_deposits': True, 'tags': 'portfolio'}), ('cash_earnings', {'title': 'Cash Earnings', 'calc_func': 'cash_earnings.vbt.sum', 'c

Get correlation data for fact cards

In [276]:

print(f'The correlation of ELK vs ETH using minutely data is: {elk_pf.returns.vbt.corr(elk_pf.bm_returns)}')
print(f'The correlation of ELK vs ETH using daily data is: {elk_pf.daily_returns.vbt.corr(eth_minutely_data.resample("1d").returns)}')
print(f'The correlation of ELK vs BTC using daily data is: {elk_pf.daily_returns.vbt.corr(btc_daily_data.returns)}')




The correlation of ELK vs ETH using minutely data is: 0.06033116798377996
The correlation of ELK vs ETH using daily data is: 0.10278153612554115
The correlation of ELK vs BTC using daily data is: 0.15688621134157893


### Now let's analyze each sub strategy
create a separate dataframe for each different strategy

In [287]:
yos_eth_fast_trades = trades[trades['strategy']=='Yosemite ETH Fast']
seq_eth_fast_trades = trades[trades['strategy']=='Yosemite ETH Fast V3']
yos_eth_slow_trades = trades[trades['strategy']=='Yosemite ETH Slow']
seq_eth_slow_trades = trades[trades['strategy']=='Yosemite ETH Slow V3']

# yos_eth_fast_trades.head(10)

In [297]:
seq_eth_fast_trades.head(10)

,id,acccount_id,strategy,position_id,symbol,direction,quantity,price,fees,trade_direction,trade_quantity
datetime,,,,,,,,,,,
2023-12-02 14:51:41.310000+00:00,2,Elk1,Yosemite ETH Fast V3,57a538f5-f42e-4a33-b345-6eb45b7e6602,ETHUSDT,open-short,0.478,2093.2,0.270154,-1,-0.478
2023-12-02 18:00:54.880000+00:00,5,Elk1,Yosemite ETH Fast V3,57a538f5-f42e-4a33-b345-6eb45b7e6602,ETHUSDT,close-short,0.478,2115.3,0.273004,1,0.478
2023-12-02 18:00:57.182000+00:00,6,Elk1,Yosemite ETH Fast V3,122bd688-18ee-4072-b883-d68d6ed40e96,ETHUSDT,open-long,0.478,2113.8,0.272808,1,0.478
2023-12-02 18:01:32.235000+00:00,9,Elk1,Yosemite ETH Fast V3,122bd688-18ee-4072-b883-d68d6ed40e96,ETHUSDT,close-long,0.478,2119.4,0.273525,-1,-0.478
2023-12-05 02:08:30.324000+00:00,11,Elk1,Yosemite ETH Fast V3,cbf51f2d-9bd4-4ff0-b02b-d576c7b55033,ETHUSDT,open-short,0.449,2228.0,0.270104,-1,-0.449
2023-12-05 07:25:01.439000+00:00,13,Elk1,Yosemite ETH Fast V3,cbf51f2d-9bd4-4ff0-b02b-d576c7b55033,ETHUSDT,close-short,0.449,2207.8,0.267649,1,0.449
2023-12-05 17:13:48.855000+00:00,15,Elk1,Yosemite ETH Fast V3,13e9c2c8-55db-4008-93d2-382f0d416f8a,ETHUSDT,open-long,0.888,2252.0,0.539942,1,0.888
2023-12-05 17:28:38.296000+00:00,17,Elk1,Yosemite ETH Fast V3,13e9c2c8-55db-4008-93d2-382f0d416f8a,ETHUSDT,close-long,0.888,2275.1,0.545468,-1,-0.888
2023-12-06 21:33:25.866000+00:00,19,Elk1,Yosemite ETH Fast V3,96c952a6-23b6-4507-966f-a2cde8b82379,ETHUSDT,open-short,0.445,2246.6,0.269928,-1,-0.445


In [ ]:

print('Yosemite ETH Fast \n')
avg_yos_eth_fast_freq = yos_eth_fast_trades.index.to_series().diff().mean()

yos_eth_fast_pf = vbt.Portfolio.from_orders(
    close = eth_minutely_data.close,
    size = yos_eth_fast_trades['trade_quantity'],
    price = yos_eth_fast_trades['price'],
    size_type = 'amount',
    fixed_fees = yos_eth_fast_trades['fees'],
    init_cash = 500,
    leverage = 4, 
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,
    freq = '1T',
    direction = 'Both',
)
yos_eth_fast_pf.plot().show()
display(yos_eth_fast_pf.stats())
# display(yos_eth_fast_pf.trades.records_readable)

# ---------------------------------------------------------------------------------------------
print('Sequoia ETH Fast \n')
avg_seq_eth_fast_freq = seq_eth_fast_trades.index.to_series().diff().mean()

seq_eth_fast_pf = vbt.Portfolio.from_orders(
    close = eth_minutely_data.close,
    size = seq_eth_fast_trades['trade_quantity'],
    price = seq_eth_fast_trades['price'],
    size_type = 'amount',
    fixed_fees = seq_eth_fast_trades['fees'],
    init_cash = 1000,
    freq = '1T',
    direction = 'Both',
    leverage = 4,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,
)
seq_eth_fast_pf.plot().show()
display(seq_eth_fast_pf.stats())
# display(seq_eth_fast_pf.trades.records_readable)

# ---------------------------------------------------------------------------------------------
print('Yosemite ETH Slow \n')
avg_yos_eth_slow_freq = yos_eth_slow_trades.index.to_series().diff().mean()

yos_eth_slow_pf = vbt.Portfolio.from_orders(
    close = eth_minutely_data.close,
    size = yos_eth_slow_trades['trade_quantity'],
    price = yos_eth_slow_trades['price'],
    size_type = 'amount',
    fixed_fees = yos_eth_slow_trades['fees'],
    init_cash = 500,
    freq = '1T',
    direction = 'Both',
    leverage = 4,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,
)
yos_eth_slow_pf.plot().show()
display(yos_eth_slow_pf.stats())
# display(yos_eth_slow_pf.trades.records_readable)

# ---------------------------------------------------------------------------------------------
print('Sequoia ETH Slow \n')
avg_seq_eth_slow_freq = seq_eth_slow_trades.index.to_series().diff().mean()

seq_eth_slow_pf = vbt.Portfolio.from_orders(
    close = eth_minutely_data.close,
    size = seq_eth_slow_trades['trade_quantity'],
    price = seq_eth_slow_trades['price'],
    size_type = 'amount',
    fixed_fees = seq_eth_slow_trades['fees'],
    init_cash = 500,
    freq = '1T',
    direction = 'Both',
    leverage = 4,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,
)
seq_eth_slow_pf.plot().show()
display(seq_eth_slow_pf.stats())
# display(seq_eth_slow_pf.trades.records_readable)




In [312]:
# Create a correlation matrix for all the strategies and the ETH price and one another
# Note, we are using the minutely data for ETH
combined_returns = pd.DataFrame({
    'elk_pf': elk_pf.returns,
    'seq_eth_fast': seq_eth_fast_pf.returns,
    'seq_eth_slow': seq_eth_slow_pf.returns,
    'yos_eth_fast': yos_eth_fast_pf.returns,
    'yos_eth_slow': yos_eth_slow_pf.returns,
    'eth_price': eth_minutely_data.returns
})

# Compute the correlation matrix
correlation_matrix = combined_returns.corr()

# Display the correlation matrix
display(correlation_matrix)
correlation_matrix.vbt.heatmap().show()


,elk_pf,seq_eth_fast,seq_eth_slow,yos_eth_fast,yos_eth_slow,eth_price
elk_pf,1.000000,0.908987,0.408607,0.829180,0.409685,0.061219
seq_eth_fast,0.908987,1.000000,0.026126,0.828527,0.028268,0.045286
seq_eth_slow,0.408607,0.026126,1.000000,0.034361,0.945198,0.063275
yos_eth_fast,0.829180,0.828527,0.034361,1.000000,0.038888,0.022906
yos_eth_slow,0.409685,0.028268,0.945198,0.038888,1.000000,0.061005
eth_price,0.061219,0.045286,0.063275,0.022906,0.061005,1.000000


For analysis you can compare the orders and or trades to the original trades dataframe

In [161]:
# display(trades)
# elk_pf.orders.records_readable
# elk_pf.trades.records_readable

In [ ]:
# Create a combined portfolio sim
# First create a combined orders dataframe from the original dataframe
combined_orders = pd.concat([yos_eth_fast_trades, seq_eth_fast_trades, yos_eth_slow_trades, seq_eth_slow_trades])
combined_orders


### Use quantstats reports

In [11]:
# Need to combine the minutely dataframe with the trades dataframe
combined_df = pd.concat([eth_minutely_data.close, trades], axis=1)
combined_df = combined_df.sort_index()


In [12]:
combined_df.head(10)

,Close,id,acccount_id,strategy,position_id,symbol,direction,quantity,price,fees,trade_direction,trade_quantity
2023-12-01 00:00:00+00:00,2052.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:01:00+00:00,2051.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:02:00+00:00,2049.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:03:00+00:00,2050.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:04:00+00:00,2050.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:05:00+00:00,2050.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:06:00+00:00,2050.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:07:00+00:00,2049.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:08:00+00:00,2049.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 00:09:00+00:00,2048.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create a QuantStats HTML Report based on the portfolio performance. Note these calculations are not taking trades into account, they are simply analyzing up days and down days etc. Most of the stats are accurate though.